In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install nlpaug
import nlpaug.augmenter.word as naw
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 23.1 MB/s eta 0:00:00


In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd, torch, re
from faker import Faker
from transformers import MarianTokenizer, MarianMTModel

fake = Faker()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def load_pair(src2tgt, tgt2src):
    tok_s = MarianTokenizer.from_pretrained(src2tgt)
    mod_s = MarianMTModel.from_pretrained(src2tgt).to(device)
    tok_t = MarianTokenizer.from_pretrained(tgt2src)
    mod_t = MarianMTModel.from_pretrained(tgt2src).to(device)
    return (tok_s, mod_s, tok_t, mod_t)

pairs = {
    'de': load_pair("Helsinki-NLP/opus-mt-en-de", "Helsinki-NLP/opus-mt-de-en"),
    'fr': load_pair("Helsinki-NLP/opus-mt-en-fr", "Helsinki-NLP/opus-mt-fr-en"),
}


def back_translate(text, lang, max_len=512):
    tok_s, mod_s, tok_t, mod_t = pairs[lang]
    mid = mod_s.generate(**tok_s(text, return_tensors='pt',
                                 truncation=True, padding=True).to(device),
                         num_beams=4, max_length=max_len, early_stopping=True)
    mid_txt = tok_s.decode(mid[0], skip_special_tokens=True)
    out = mod_t.generate(**tok_t(mid_txt, return_tensors='pt',
                                 truncation=True, padding=True).to(device),
                         num_beams=4, max_length=max_len, early_stopping=True)
    return tok_t.decode(out[0], skip_special_tokens=True)


pat = re.compile(r'^\s*(Job Title|Abstract|Employer|Location|Highlights)\s*:\s*(.*)$', re.I)

def split(txt):
    fields, tail, in_tail = {}, [], False
    for ln in txt.splitlines():
        if ln.lower().startswith('contents:'):
            in_tail = True
        if not in_tail:
            m = pat.match(ln)
            if m:
                fields[m.group(1).title()] = m.group(2).strip()
        if in_tail:
            tail.append(ln)
    return fields, '\n'.join(tail)

def build(fields, tail):
    order = ["Job Title", "Abstract", "Employer", "Location", "Highlights"]
    head = '\n'.join(f"{o}: {fields[o]}" for o in order if o in fields)
    return head + ('\n' + tail.lstrip('\n') if tail else '')


df = pd.read_csv('./drive/MyDrive/AdSeek/Preprocessing/After_cleaning/work_training.csv')
rows = []

for _, r in df.iterrows():
    base, tail = split(r['prompt'])
    for lang in ('de', 'fr'):
        f = base.copy()
        f['Employer'] = fake.company()
        if f.get('Location') and f['Location'] != '-':
            f['Location'] = fake.city()
        for fld in ('Job Title', 'Abstract', 'Highlights'):
            if fld in f:
                try:
                    f[fld] = back_translate(f[fld], lang)
                except Exception as e:
                    print(f"Skip wrong chars: {fld} | {e}")
        new_prompt = build(f, tail)
        rows.append({'prompt': new_prompt, 'complete': r['complete']})

df_bt = pd.DataFrame(rows)
df_final = pd.concat([df, df_bt], ignore_index=True)
df_final.to_csv('work_training_aug.csv', index=False)
print(f"Augment completed: {len(df_final)} ,from {len(df)} to {len(df_bt)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

Augment completed: 297 ,from 99 to 198


In [3]:
def augment_to_5(df, label_col='complete', text_col='prompt'):

    new_rows = []

    label_counts = df[label_col].value_counts()
    for label, count in label_counts.items():
        if count < 5:
            need = 5 - count
            examples = df[df[label_col] == label]

            for _ in range(need):
                row = examples.sample(1).iloc[0].copy()
                row[text_col] = aug.augment(row[text_col])
                new_rows.append(row)

    if new_rows:
        df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

    return df

In [5]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

salary_df = pd.read_csv('./drive/MyDrive/AdSeek/Preprocessing/After_cleaning/salary_training.csv')
seniority_df = pd.read_csv('./drive/MyDrive/AdSeek/Preprocessing/After_cleaning/seniority_training.csv')

aug = naw.SynonymAug(aug_src='wordnet', aug_min=0, aug_max=1)
salary_aug = augment_to_5(salary_df)
seniority_aug = augment_to_5(seniority_df)

salary_aug.to_csv('salary_training_augmented.csv', index=False)
seniority_aug.to_csv('seniority_training_augmented.csv', index=False)


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
